<img src="https://news.illinois.edu/files/6367/543635/116641.jpg" alt="University of Illinois" width="250"/>

# PyTorch implementation of Minimax #
By Richard Sowers
* <r-sowers@illinois.edu>
* <https://publish.illinois.edu/r-sowers/>

Copyright 2022 University of Illinois Board of Trustees. All Rights Reserved.

In [ ]:
import os
import numpy
import pandas
import time
import random
import matplotlib
#%matplotlib notebook
import matplotlib.pyplot as plt
import scipy.stats
#from pandas.plotting import autocorrelation_plot
import matplotlib.offsetbox as offsetbox
from matplotlib.ticker import StrMethodFormatter
import sklearn.linear_model
import sklearn.model_selection
import itertools


def saver(fname):
    plt.savefig(fname+".png",bbox_inches="tight")

def legend(pos="bottom",ncol=3):
    if pos=="bottom":
        plt.legend(bbox_to_anchor=(0.5,-0.2), loc='upper center',facecolor="lightgray",ncol=ncol)
    elif pos=="side":
        plt.legend(bbox_to_anchor=(1.1,0.5), loc='center left',facecolor="lightgray",ncol=1)

def textbox(txt,fname=None):
    plt.figure(figsize=(1,1))
    plt.gca().add_artist(offsetbox.AnchoredText("\n".join(txt), loc="center",prop=dict(size=30)))
    plt.axis('off')
    if fname is not None:
        saver(fname)
    plt.show()
    plt.close()

In [ ]:
import torch
import numpy

In [ ]:
#for some reason, this needs to be in a separate cell
params={
    "font.size":15,
    "lines.linewidth":5
}
plt.rcParams.update(params)

In [ ]:
pngfiles=[f for f in os.listdir(".") if f.endswith(".png")]
print("existing png files: "+str(pngfiles))
#print([os.remove(f) for f in pngfiles])

existing png files: []


In [ ]:
def getfile(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pandas.read_csv(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pandas.read_csv(loc,**kwargs)
    return out

# configuration variables

# main

In [ ]:
class VariableWrapper(torch.nn.Module):
    #degenerate implementation of Module; forward returns the variable
    def __init__(self,SEED=0):
        super().__init__() #run init of torch.nn.Module
        if SEED is not None:
          torch.manual_seed(SEED)
        self.variable = torch.nn.Parameter(torch.rand(1), requires_grad=True)

    def train(self,flag: bool):
        self.requires_grad_(flag)

    def get(self):
        return self.variable.data.item()

    def forward(self):
        return self.variable

X=VariableWrapper()
Y=VariableWrapper()

In [ ]:
def f(x,y):
  return (x-2*y)**2-7*(y-1)**2
  # optimum is x=2, y=1

optimizer = torch.optim.Adam(list(X.parameters())+list(Y.parameters()))

In [ ]:
MAX_iter_outer=250
MAX_iter_inner=10

for ctr in range(MAX_iter_outer+1):

  # minimize in X, hold Y constant
  X.train(True)
  Y.train(False)
  for ctrprime in range(MAX_iter_inner):
    optimizer.zero_grad()
    (f(X(),Y())).backward() #evaluate loss and then backpropagate
    optimizer.step()

  # maximize in Y, hold X constant
  X.train(False)
  Y.train(True)
  for ctrprime in range(MAX_iter_inner):
    optimizer.zero_grad()
    (-f(X(),Y())).backward() #evaluate negative loss and then backpropagate
    optimizer.step()

  loss=f(X(),Y())
  if ctr%int(MAX_iter_outer/10)==0: #print out data for 10 intermediate steps
      print("iteration {}: loss={:.3f},X={:.3f},Y={:.3f}".format(ctr,loss.item(),X.get(),Y.get()),)


iteration 0: loss=-1.456,X=0.512,Y=0.506
iteration 25: loss=0.313,X=0.895,Y=0.817
iteration 50: loss=0.616,X=1.291,Y=1.042
iteration 75: loss=0.272,X=1.645,Y=1.152
iteration 100: loss=-0.003,X=1.912,Y=1.142
iteration 125: loss=-0.020,X=2.045,Y=1.061
iteration 150: loss=0.002,X=2.042,Y=0.997
iteration 175: loss=-0.000,X=2.002,Y=0.992
iteration 200: loss=0.000,X=1.997,Y=1.000
iteration 225: loss=-0.000,X=2.000,Y=1.000
iteration 250: loss=0.000,X=2.000,Y=1.000
